In [1]:
import os
import cv2
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import numpy as np

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

dataset_folder = "final2/"
CLASSES = 6
batch_size = 32

In [2]:

# Haar Cascade model for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Transformation pipeline for PyTorch
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Dataset
class FaceDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        for label, person_folder in enumerate(os.listdir(root_dir)):
            person_folder_path = os.path.join(root_dir, person_folder)
            for filename in os.listdir(person_folder_path):
                image_path = os.path.join(person_folder_path, filename)
                self.samples.append((image_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = cv2.imread(image_path)
        if image is not None:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))

            if len(faces) > 0:
                x, y, w, h = faces[0]
                face = image[y:y+h, x:x+w]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face = Image.fromarray(face)
            else:
                face = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                face = Image.fromarray(face)

            if self.transform:
                face = self.transform(face)
            return face, label
        else:
            return None, None

dataset = FaceDataset(dataset_folder, transform=transform)
train_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, stratify=[label for _, label in dataset.samples if _ is not None])
train_subset = torch.utils.data.Subset(dataset, train_indices)
test_subset = torch.utils.data.Subset(dataset, test_indices)

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

# Model configuration
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, CLASSES)

# Freeze the feature layers
for param in model.features.parameters():
    param.requires_grad = False

# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# Training loop
def train_epoch(loader, model, criterion, optimizer):
    model.train()
    total_loss = 0
    for images, labels in loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Test loop
def test(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Run training and testing
for epoch in range(10):
    train_loss = train_epoch(train_loader, model, criterion, optimizer)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss}')
    scheduler.step()
    accuracy = test(test_loader, model)
    print(f'Test Accuracy: {accuracy:.2f}')

# Save the trained model
model_path = 'face_recognition_model.pth'
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Train Loss: 1.6050641107559205
Test Accuracy: 0.62
Epoch 2, Train Loss: 1.1528487777709961
Test Accuracy: 0.65
Epoch 3, Train Loss: 0.9254897165298462
Test Accuracy: 0.68
Epoch 4, Train Loss: 0.7867285776138305
Test Accuracy: 0.75
Epoch 5, Train Loss: 0.7121101117134094
Test Accuracy: 0.72
Epoch 6, Train Loss: 0.6512606370449067
Test Accuracy: 0.78
Epoch 7, Train Loss: 0.5821757602691651
Test Accuracy: 0.77
Epoch 8, Train Loss: 0.5260961019992828
Test Accuracy: 0.79
Epoch 9, Train Loss: 0.5130953156948089
Test Accuracy: 0.78
Epoch 10, Train Loss: 0.5181395280361175
Test Accuracy: 0.80
Model saved to face_recognition_model.pth
